In [2]:
__author__ = 'Wzhou9'

In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn import cross_validation
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

In [4]:
train =pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# Add target attribute "loss" in the test dataset and fulfill with Null
test["loss"]=np.nan

In [9]:
 joined = pd.concat([train, test])

In [10]:
def remove_st(col1,col2):
    set_1 = set(col1.unique())
    set_2 = set(col2.unique())
    rm = (set_2 - set_1).union(set_1 - set_2) 
    return rm

In [11]:
#df = pd.DataFrame([{'a': 15, 'b': 15, 'c': 5}, {'a': 20, 'b': 10, 'c': 7}, {'a': 25, 'b': 30, 'c': 9}])
#remove_st(df['a'],df['b'])


In [12]:
def filter_rm(col,rm):
    if col in rm:
        return np.nan
    return col
    

In [13]:
def handle_cat(train,test):
    #join two datasets
    joinDS = pd.concat([train, test])
    headercat = joinDS.select_dtypes(include = ['object']).columns
    #
    for i in headercat:
        #find remove set
        rm_list = remove_st(train[i], test[i])
        joinDS[i] = joinDS[i].apply(lambda x: filter_rm(x,rm_list), 1)
        joinDS[i] = pd.factorize(joinDS[i].values, sort=True)[0]    
    return joinDS   



In [14]:
joinDS = handle_cat(train,test)

In [15]:
# taken from Ali's script (https://www.kaggle.com/aliajouz/allstate-claims-severity/singel-model-lb-1117)
joinDS["cont1"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont1"]))
joinDS["cont4"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont4"]))
joinDS["cont5"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont5"]))
joinDS["cont8"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont8"]))
joinDS["cont10"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont10"]))
joinDS["cont11"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont11"]))
joinDS["cont12"] = np.sqrt(preprocessing.minmax_scale(joinDS["cont12"]))

joinDS["cont6"] = np.log(preprocessing.minmax_scale(joinDS["cont6"]) + 0000.1)
joinDS["cont7"] = np.log(preprocessing.minmax_scale(joinDS["cont7"]) + 0000.1)
joinDS["cont9"] = np.log(preprocessing.minmax_scale(joinDS["cont9"]) + 0000.1)
joinDS["cont13"] = np.log(preprocessing.minmax_scale(joinDS["cont13"]) + 0000.1)
joinDS["cont14"] = (np.maximum(joinDS["cont14"] - 0.179722, 0) / 0.665122) ** 0.25

In [16]:
train_a = joinDS[joinDS['loss'].notnull()]
test_a = joinDS[joinDS['loss'].isnull()]

In [17]:
train_a.shape

(188318, 132)

In [18]:
#transform loss, input of xgboost
shift = 200
train_x = train_a.drop(['loss','id'],1)
train_label = np.log(train_a['loss'] + shift)


    

In [19]:
test_x = test_a.drop(['id','loss'],1)
test_id = test['id']

In [37]:
train_x.to_csv('train_x.csv', index = False)

In [38]:
train_label.to_csv("train_label.csv", index = False)

In [39]:
test_x.to_csv('test_x.csv', index = False)

In [40]:
test_id.to_csv('test_id.csv', index = False)

In [52]:
train_x = pd.read_csv("train_x.csv")
train_label = pd.read_csv("train_label.csv",header = None)
test_x = pd.read_csv("test_x.csv")
test_id = pd.read_csv("test_id.csv", header = None)


In [53]:
train_label.head()

,0
0,7.788701
1,7.302227
2,8.072495
3,7.038652
4,7.994244


In [49]:
test_x.shape

(125546, 130)

In [50]:
test_id.shape

(125545, 1)

In [51]:
test_id.head()

,4
0,6
1,9
2,12
3,15
4,17
